# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [10]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [11]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [43]:
prodQuantity=data.groupby(by=['ProductID','CustomerID'],as_index=False).sum()
prodQuantity.head()

,ProductID,CustomerID,SalesID,Quantity
0,1,639,117790,1
1,1,1428,5818988,1
2,1,4595,2634526,2
3,1,5230,2997936,2
4,1,7772,4855697,2


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [93]:
table=prodQuantity.pivot('CustomerID','ProductID','Quantity')
table=table.fillna(0)
print(table.shape)
table.head()

(1000, 452)


ProductID,1,2,3,4,5,6,7,8,9,10,...,443,444,445,446,447,448,449,450,451,452
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
264,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
412,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [97]:
distances=pdist(table,'euclidean')
distancesMarix=squareform(distances)
distances = pd.DataFrame(1/(1 + distancesMarix), index=table.index, columns=table.index)
print(distances.shape)
#display(distances)


(1000, 1000)


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [99]:
nearCustomerID=distances[33].sort_values(ascending=False)[1:6].index
for customerID in nearCustomerID:
    print(data.loc[customerID].FirstName,data.loc[customerID].LastName)


Kimberly Stewart
Jerrod Pierce
Rafael Osborne
Arturo O'Connor
Lorraine Rubio


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [103]:
near33=table.loc[list(nearCustomerID)]

    
near33.head(5)

ProductID,1,2,3,4,5,6,7,8,9,10,...,443,444,445,446,447,448,449,450,451,452
CustomerID,,,,,,,,,,,,,,,,,,,,,
264,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3317,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2503,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [111]:
totalBuyProducts=near33.sum()

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [155]:
decision_Serie=pd.concat([table.loc[33], totalBuyProducts], axis=1)
decision_Serie=decision_Serie[decision_Serie[33]==0]
decision_Serie.columns=['target','near']
products=list(decision_Serie.near.sort_values(ascending=False)[:5].index)
for product in products:
    print(data.loc[product].ProductName)

Cheese Cloth No 100
Arizona - Green Tea
Beer - Original Organic Lager
Arizona - Green Tea
Bread - Hot Dog Buns


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [166]:
def get5Best(CustomerID, distances):
    
    nearCustomerID=distances[CustomerID].sort_values(ascending=False)[1:6].index
    
    near=table.loc[list(nearCustomerID)]
    totalBuyProducts=near.sum()
    
    decision_Serie=pd.concat([table.loc[CustomerID], totalBuyProducts], axis=1)
    decision_Serie=decision_Serie[decision_Serie[CustomerID]==0]
    decision_Serie.columns=['target','near']
    return list(decision_Serie.near.sort_values(ascending=False)[:5].index)


targets=[33,77352,23548]
recomendations={}

for target in targets:
    recomendations[target]=get5Best(target,distances)
    
pd.DataFrame(recomendations)
    
   
    

,33,77352,23548
0,269,296,70
1,382,366,42
2,442,328,386
3,402,129,178
4,61,350,349


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [167]:
distances_cosine=pdist(table,'cosine')
distancesMarix_cosine=squareform(distances_cosine)
distances_cosine = pd.DataFrame(1/(1 + distancesMarix_cosine), index=table.index, columns=table.index)

recomendations_cosine={}

for target in targets:
    recomendations_cosine[target]=get5Best(target,distances_cosine)
    
pd.DataFrame(recomendations_cosine)

,33,77352,23548
0,167,337,438
1,250,387,262
2,3,194,340
3,141,50,355
4,304,107,218
